In [2]:
# Boiler plate stuff to start the module
import jpype
import jpype.imports
from jpype.types import *
import sys 
import numpy as np
import traceback
import random
import Client
import copy
import threading
import queue
import operator

C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
# import the Java modules
from de.fhac.mazenet.server.generated import AwaitMoveMessageData
from de.fhac.mazenet.server.generated import Errortype
from de.fhac.mazenet.server.generated import MazeCom
from de.fhac.mazenet.server.generated import MazeComMessagetype
#from de.fhac.mazenet.server.generated import MoveMessageData
from de.fhac.mazenet.server.generated import BoardData

from de.fhac.mazenet.server.networking import MazeComMessageFactory

from de.fhac.mazenet.server.game import Board
#from de.fhac.mazenet.server.game import Position
#from de.fhac.mazenet.server.game import Card
#from mazeCom import AwaitMoveMessageData, Errortype, MazeCom, MazeComMessagetype, MoveMessageData, BoardData
#from Boardpy import Board
from MoveMessageDatapy import MoveMessageData
from Positionpy import Position
from Cardpy import Card

In [4]:
HOST = "127.0.0.1"
PORT = 9888

In [5]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random
import os

In [6]:
from mss import mss
import pydirectinput
import cv2
import pytesseract
import matplotlib.pyplot as plt
import time

In [7]:
ls_range = np.arange(1,6,2)
potentialShiftMoves =  []
potentialShiftMovesPos =  []
for i in ls_range:
    potentialShiftMoves.append((0, i))
    potentialShiftMoves.append((6, i))
    potentialShiftMoves.append((i, 0))
    potentialShiftMoves.append((i, 6))
    potentialShiftMovesPos.append(Position(row=0, col=i))
    potentialShiftMovesPos.append(Position(row=6, col=i))
    potentialShiftMovesPos.append(Position(row=i, col=0))
    potentialShiftMovesPos.append(Position(row=i, col=6))

In [8]:
import functools

from pettingzoo import AECEnv, ParallelEnv
from pettingzoo.utils import agent_selector, parallel_to_aec, wrappers

In [9]:
class Agent_Client:
    def __init__(self, name, client=None):
        self.name = name
        self.client = client

In [10]:
class MARL_Env_Parallel(ParallelEnv):
    metadata = {"render_modes": ["human"], "name": "rps_v2"}

    def __init__(self, render_mode=None, num_players=4):
        #self.manager = Client.Client("manager", HOST, PORT, True)
        #self.manager.login_manager()
        
        self.num_players = num_players
        self.possible_agents = [Agent_Client(name="player_" + str(r)) for r in range(self.num_players)]
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )

        # Gym spaces are defined and documented here: https://gym.openai.com/docs/#spaces
        self._action_spaces = {agent: MultiDiscrete([12, 4, 7, 7], dtype=np.int8) for agent in self.possible_agents}
        self._observation_spaces = {
            agent: Tuple((Box(low=0, high=6, shape=(1,2), dtype=np.uint8),
                           Box(low=0, high=6, shape=(1,2), dtype=np.uint8)
                           ))  for agent in self.possible_agents
        }
        self.render_mode = render_mode
        self.player_pos = {}
        self.next_treasure_pos = {}
        self.is_done = False
        #self.manager = Client.Client("manager", HOST, PORT, True)
        #self.manager.login_manager()

    # this cache ensures that same space object is returned for the same agent
    # allows action space seeding to work as expected
    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        # gymnasium spaces are defined and documented here: https://gymnasium.farama.org/api/spaces/
        return Tuple((Box(low=0, high=6, shape=(1,2), dtype=np.uint8),
                           Box(low=0, high=6, shape=(1,2), dtype=np.uint8)
                           ))
    
    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return MultiDiscrete([12, 4, 7, 7], dtype=np.int8)

    def render(self):
        """
        Renders the environment. In human mode, it can print to terminal, open
        up a graphical window, or open up some other display that a human can see and understand.
        """
        pass

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def moeglicheOrientierungen(self, rot, card):
        card_ = Card(c=card)
        openings = card_.getOpenings()
        print("card.getShape:", type(card.getShape()))
        if(card.getShape() == Card.CardShape.I):
            openings.setBottom(not openings.isBottom())
            openings.setLeft(not openings.isLeft())
            openings.setRight(not openings.isRight())
            openings.setTop(not openings.isTop())
            return  Card(c=card_)
        else:
            if(rot == 0):
                return Card(None, card.getShape(), Card.Orientation.D0, card.getTreasure())
            if(rot == 1):
                return Card(None, card.getShape(), Card.Orientation.D90, card.getTreasure())
            if(rot == 2):
                return Card(None, card.getShape(), Card.Orientation.D180, card.getTreasure())
            if(rot == 3):
                return Card(None, card.getShape(), Card.Orientation.D270, card.getTreasure())
        return Card(None, card.getShape(), Card.Orientation.D0, card.getTreasure())
    
    def Move(self, awaitMove, action, agent):
        reward = 0
        boardData = awaitMove.getBoard()
        treasure = awaitMove.getTreasureToFindNext()
        board = Board(boardData)
        playerPosition = board.findPlayer(agent.client.getId())
        psm = potentialShiftMoves[action[0]]
        position = Position(row=psm[0], col=psm[1])
        
        potentialMove = MoveMessageData()
        psmp = copy.copy(potentialShiftMovesPos)
        if(position == board.getForbidden()):
            #print("forbidden shift position, random shift position used, reward decreased")
            reward -= 10
            try:
                psmp.remove(position)
            except Exception:
                print("element not in list, skipping")
            potentialMove.setShiftPosition(psmp[0])
        else:
            potentialMove.setShiftPosition(position)
        
        treasurePositionData = board.findTreasure(treasure)
        
        orientedShiftCard = self.moeglicheOrientierungen(action[1], board.getShiftCard())
        
        potentialMove.setShiftCard(orientedShiftCard)
        potentialMove.setNewPinPos(playerPosition)

        boardNext = board.fakeShift(potentialMove)
        treasurePositionData = boardNext.findTreasure(treasure)
        
        new_player_pos = boardNext.findPlayer(agent.client.getId())
        reachablePositions = boardNext.getAllReachablePositions(new_player_pos)
        rp = np.array(reachablePositions)
        chosen_pos_obj = Position(row=action[2], col=action[3])
        chosen_pos_arr = np.array(action[2], action[3])
        if(treasurePositionData == None):
            #print("no treasure? skipping")
            potentialMove.setNewPinPos(new_player_pos)
            self.next_treasure_pos[agent] = treasurePositionData
            self.player_pos[agent] = new_player_pos
            return potentialMove, reward

        if(chosen_pos_obj not in rp):
            #print("cannot reach position, reward decreased")
            potentialMove.setNewPinPos(new_player_pos)
            self.next_treasure_pos[agent] = treasurePositionData
            self.player_pos[agent] = new_player_pos
            return potentialMove, reward - 10
        else:
            #print("treasure position data:", treasurePositionData)
            treasurePosition = Position(treasurePositionData)
            #print("treasure pos:", treasurePosition)

            if(chosen_pos_obj == treasurePosition):
                #print("straight to the treasure, reward increased!")
                potentialMove.setNewPinPos(treasurePosition)
                self.next_treasure_pos[agent] = treasurePositionData
                self.player_pos[agent] = treasurePosition
                return potentialMove, reward + 20
            else:
                #print("reachable position, but no treasure")
                potentialMove.setNewPinPos(chosen_pos_obj)
                self.next_treasure_pos[agent] = treasurePositionData
                self.player_pos[agent] = chosen_pos_obj
                return potentialMove, reward - 5

        #print("Object not reachable: random move :/")
        potentialMove.setNewPinPos(reachablePositions[0])
        self.next_treasure_pos[agent] = treasurePositionData
        self.player_pos[agent] = reachablePositions[0]
        return potentialMove, reward - 10
    
    def awaitMove(self, receivedMazeCom, action, agent):
        awaitMove = receivedMazeCom.getAwaitMoveMessage()
        move, reward = self.Move(awaitMove, action, agent)
        mazeComToSend = MazeCom()
        mazeComToSend.setId(agent.client.getId())
        mazeComToSend.setMessagetype(MazeComMessagetype.MOVE)
        mazeComToSend.setMoveMessage(move)
        agent.client.out.write(mazeComToSend)
        return reward          
    
    def run_agent(self, agent, action, return_val):
        reward = 0
        #print(f"agents:", agent)
        #print(f"actions:", action)
        try:
            print("starrt rer")
            receivedMazeCom = agent.client.in_.readMazeCom()
            print("end ererer")
            if(receivedMazeCom.getMessagetype() == MazeComMessagetype.LOGINREPLY):
                newid = receivedMazeCom.getLoginReplyMessage().getNewID()
                agent.client.setId(newid)
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.ACCEPT):
                print(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                reward += 2
            elif (receivedMazeCom.getMessagetype() == MazeComMessagetype.DISCONNECT):
                print(receivedMazeCom.getDisconnectMessage().getErrortypeCode())
                reward -= 10
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.AWAITMOVE):
                print(action)
                reward += self.awaitMove(receivedMazeCom, action, agent)
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.MOVEINFO):
                #print("in MoveInfo")
                x = 0
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.WIN):
                print("You have won")
                self.is_done = True
                reward += 100
            else:
                print("Unknown message type: " + receivedMazeCom.getMessagetype())
        except Exception as e:
            print("exception time")
            print(traceback.format_exc())
            if str(type(e)) == "<java class 'java.net.SocketException'>":
                print("socket error, disconnected")
                sys.exit(1)
            elif str(type(e)) == "<java class 'java.io.EOFException'>":
                print("end reached, disconnecting")
                sys.exit(1)
                
        observations = (self.player_pos[agent], self.next_treasure_pos[agent])
        if self.is_done:
            terminations = True
        else:
            terminations = False
        truncations = False
        infos = {}
        return_val[0] = observations
        return_val[1] = reward
        return_val[2] = terminations
        return_val[3] = truncations
        return_val[4] = infos
    
    def reset(self, seed=None, return_info=False, options=None):
        """
        Reset needs to initialize the `agents` attribute and must set up the
        environment so that render(), and step() can be called without issues.
        Here it initializes the `num_moves` variable which counts the number of
        hands that are played.
        Returns the observations for each agent
        """
        pydirectinput.click(x=1050, y=135)
        # start
        pydirectinput.click(x=950, y=135)
        #print("t1")
        #self.manager.stop_game(self.num_players)
        #self.manager.start_game(self.num_players)
        #print("t2")
        #time.sleep(1)
        #for agent in self.possible_agents:
        #    agent.client = Client.Client(agent.name, HOST, PORT, True)
        #"""
        for agent in self.possible_agents:
            while True:
                agent.client = Client.Client(agent.name, HOST, PORT, True)
                try:
                    login = MazeComMessageFactory.createLoginMessage(agent.name)
                    agent.client.out.write(login)
                    receivedMazeCom = agent.client.in_.readMazeCom()
                    if(receivedMazeCom.getMessagetype() == MazeComMessagetype.LOGINREPLY):
                        newid = receivedMazeCom.getLoginReplyMessage().getNewID()
                        print("newid:", newid)
                        agent.client.setId(newid)
                        break
                    else:
                        print(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                except:
                    print("failed login :)")
        
        #self.possible_agents = [Client.Client("player_" + str(r), HOST, PORT, True) for r in range(self.num_players+1)]
        #self.possible_agents = self.possible_agents[1:]
        
        self.is_done = False
        for i in self.possible_agents:
            print("id:", i.client.id_)
        #print("agents:", self.agents)
        self.possible_agents = sorted(self.possible_agents, key=operator.attrgetter('client.id_'))
        for i in self.possible_agents:
            print("id:", i.client.id_)
        #"""
        self.agents = self.possible_agents[:]
        observations = {agent: None for agent in self.agents}
        print("reset done")
        if not return_info:
            return observations
        else:
            infos = {agent: {} for agent in self.agents}
            return observations, infos

    def step(self, actions):
        """
        step(action) takes in an action for each agent and should return the
        - observations
        - rewards
        - terminations
        - truncations
        - infos
        dicts where each dict looks like {agent_1: item_1, agent_2: item_2}
        """
        # If a user passes in actions with no agents, then just return empty observations, etc.
        if not actions:
            self.agents = []
            return {}, {}, {}, {}, {}
        
        i = 0
        return_val= [[None]*5]*self.num_players
        for agent in self.agents:
            try:
                self.run_agent(agent, actions[agent], return_val[i])
                i += 1
            except:
                print(traceback.format_exc())
                print("dang")
        
        observations, rewards, terminations, truncations, infos = {}, {}, {}, {}, {}
        
        for t in range(self.num_players):
            observations[self.agents[t]] = return_val[t][0]
            rewards[self.agents[t]] = return_val[t][1]
            terminations[self.agents[t]] = return_val[t][2]
            truncations[self.agents[t]] = return_val[t][3]
            infos[self.agents[t]] = return_val[t][4]
        print("we're done")
        # rewards for all agents are placed in the rewards dictionary to be returned
        return observations, rewards, terminations, truncations, infos

In [11]:
from pettingzoo.test import api_test
from pettingzoo.test import parallel_api_test

In [12]:
Card.CardShape.I.name

str

In [ ]:
parallel_env = MARL_Env_Parallel()
print("did it?")
observations = parallel_env.reset()
print("tigt")
for step in range(1000):
    actions = {agent: parallel_env.action_space(agent).sample() for agent in parallel_env.agents}  # this is where you would insert your policy
    observations, rewards, terminations, truncations, infos = parallel_env.step(actions)

did it?
newid: 4
newid: 2
newid: 3
newid: 1
id: 4
id: 2
id: 3
id: 1
id: 1
id: 2
id: 3
id: 4
reset done
tigt
starrt rer
end ererer
[4 2 0 4]
card.getShape: <java class 'de.fhac.mazenet.server.game.Card.CardShape'>
exception time
Traceback (most recent call last):
  File "C:\Users\AStA\AppData\Local\Temp\ipykernel_8120\118066941.py", line 180, in run_agent
    reward += self.awaitMove(receivedMazeCom, action, agent)
  File "C:\Users\AStA\AppData\Local\Temp\ipykernel_8120\118066941.py", line 153, in awaitMove
    move, reward = self.Move(awaitMove, action, agent)
  File "C:\Users\AStA\AppData\Local\Temp\ipykernel_8120\118066941.py", line 101, in Move
    orientedShiftCard = self.moeglicheOrientierungen(action[1], board.getShiftCard())
  File "C:\Users\AStA\AppData\Local\Temp\ipykernel_8120\118066941.py", line 60, in moeglicheOrientierungen
    if(card.getShape() == Card.CardShape.I):
SystemError: Fail in conversion


Traceback (most recent call last):
  File "C:\Users\AStA\AppData\Local\T

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

import supersuit as ss
from ray.tune.registry import register_env
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray import tune

In [ ]:
env = MARL_Env_Parallel(4)
#env_steps = 1000  # 2 * env.width * env.height  # Code uses 1.5 to calculate max_steps
#rollout_fragment_length = 50
env = ss.pettingzoo_env_to_vec_env_v1(env)
env = ss.concat_vec_envs_v1(env, 1, num_cpus=1, base_class='stable_baselines3')

In [ ]:
model = PPO("MlpPolicy", env, tensorboard_log="./logs", verbose=3, #gamma=0.95, 
    #n_steps=rollout_fragment_length, ent_coef=0.01, 
    #learning_rate=5e-5, vf_coef=1, max_grad_norm=0.9, gae_lambda=1.0, n_epochs=30, clip_range=0.3,
    #batch_size=150, seed=seed)
           )

In [ ]:
asy

In [ ]:
def env_creator(args):
    env = MARL_Env_Parallel(4)
    return env

In [ ]:
env_name = "marl_env_para"

register_env(env_name, lambda config: ParallelPettingZooEnv(env_creator(config)))

test_env = ParallelPettingZooEnv(env_creator({}))
obs_space = test_env.observation_space
act_space = test_env.action_space

In [ ]:
def gen_policy(i):
    config = {
        "gamma": 0.99,
    }
    return (None, obs_space, act_space, config)

In [ ]:
policies = {"policy_0": gen_policy(0)}
policy_ids = list(policies.keys())

In [ ]:
tune.run(
    "PPO",
    name="PPO",
    stop={"timesteps_total": 5000000},
    checkpoint_freq=10,
    local_dir="~/ray_results/"+env_name,
    config={
        # Environment specific
        "env": env_name,
        # https://github.com/ray-project/ray/issues/10761
        "no_done_at_end": True,
        # "soft_horizon" : True,
        "num_gpus": 0,
        "num_workers": 4,
        "num_envs_per_worker": 1,
        "compress_observations": False,
        "batch_mode": 'truncate_episodes',
        "clip_rewards": False,
        "vf_clip_param": 500.0,
        "entropy_coeff": 0.01,
        # effective batch_size: train_batch_size * num_agents_in_each_environment [5, 10]
        # see https://github.com/ray-project/ray/issues/4628
        "train_batch_size": 1000,  # 5000
        "rollout_fragment_length": 50,  # 100
        "sgd_minibatch_size": 100,  # 500
        "vf_share_layers": False
        },
)

In [ ]:
model.learn(total_timesteps=100000)
model.save(f"policy_ppo_{train_timesteps}")

In [ ]:
env = MARL_Env_Parallel(4)

In [ ]:
path = 'Training/Logs'
# model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=path)